In [11]:
import os
import pandas as pd
import numpy as np

In [4]:
cd ..

/work-ceph/bizer-tp2021/data_integration_using_deep_learning


In [5]:
parent_path = os.path.dirname(os.getcwd()) + '/data_integration_using_deep_learning'
parent_path

'/work-ceph/bizer-tp2021/data_integration_using_deep_learning'

In [6]:
cleaned_path = parent_path + "/src/data/LocalBusiness" 
cleaned_file = '20-12-21_ClusterFile_withCleaning'

LB_top100_path =  parent_path + "/src/data/LocalBusiness/LocalBusiness_top100/geo_preprocessed_v3" 
LB_min3_path =  parent_path + "/src/data/LocalBusiness/LocalBusiness_minimum3/geo_preprocessed_v3" 
Hotel_top100_path =  parent_path + "/src/data/Hotel/Hotel_top100/geo_preprocessed_v3" 
Hotel_min3_path =  parent_path + "/src/data/Hotel/Hotel_minimum3/geo_preprocessed_v3" 
Restaurant_top100_path =  parent_path + "/src/data/Restaurant/Restaurant_top100/geo_preprocessed_v3" 
Restaurant_min3_path =  parent_path + "/src/data/Restaurant/Restaurant_minimum3/geo_preprocessed_v3" 

splitting3_path = parent_path + "/src/data/LocalBusiness/Splitting_12.20/Train_Validation_Test"
splitting2_path = parent_path + "/src/data/LocalBusiness/Splitting_12.20/Train_Test"

In [7]:
train_tables3 = pd.read_csv(os.path.join(splitting3_path, 'training tables.csv')).drop(columns='Unnamed: 0')['0'].tolist()
val_tables3 = pd.read_csv(os.path.join(splitting3_path, 'validation tables.csv')).drop(columns='Unnamed: 0')['0'].tolist()
test_tables3 = pd.read_csv(os.path.join(splitting3_path, 'testing tables.csv')).drop(columns='Unnamed: 0')['0'].tolist()

train_tables2 = pd.read_csv(os.path.join(splitting2_path, 'training tables_v2.csv')).drop(columns='Unnamed: 0')['0'].tolist()
test_tables2 = pd.read_csv(os.path.join(splitting2_path, 'testing tables_v2.csv')).drop(columns='Unnamed: 0')['0'].tolist()

In [8]:
df_cleaned = pd.read_json(os.path.join(cleaned_path, cleaned_file), compression='gzip', orient='records', lines=True)
selected_tables = df_cleaned[df_cleaned['clusterNtables']>=8]['table_id'].unique()

df_cleaned = df_cleaned[['cluster_id', 'table_id', 'row_id']]

In [9]:
source_paths = [LB_top100_path, LB_min3_path, Hotel_top100_path, Hotel_min3_path, Restaurant_top100_path, Restaurant_min3_path]
target_paths = [splitting3_path, splitting2_path]

for path in source_paths:
    files = os.listdir(path)
    for file in files:
        if not file in selected_tables:
            continue
        else:
            df = pd.read_json(os.path.join(path,file), compression='gzip', orient='records', lines=True)
            df['table_id'] = file
            df_merge = df.merge(df_cleaned, how='left', on=['table_id', 'row_id'])
            df_merge['cluster_id'].fillna(-100, inplace=True)
            df_merge = df_merge.astype({'cluster_id': 'int32'})
            
            # export the files: train_val_test splitting
            if file in train_tables3:
                df_merge.to_json(os.path.join(splitting3_path + '/train tables', str(file)), 
                                 compression='gzip', orient='records', lines=True)
            elif file in val_tables3:
                df_merge.to_json(os.path.join(splitting3_path + '/validation tables', str(file)),
                                 compression='gzip', orient='records', lines=True)
            elif file in test_tables3:
                df_merge.to_json(os.path.join(splitting3_path + '/test tables', str(file)),
                                 compression='gzip', orient='records', lines=True)
                
            # export the files: train_test splitting
            if file in train_tables2:
                df_merge.to_json(os.path.join(splitting2_path + '/train tables', str(file)),
                                 compression='gzip', orient='records', lines=True)
            elif file in test_tables2:
                df_merge.to_json(os.path.join(splitting2_path + '/test tables', str(file)),
                                 compression='gzip', orient='records', lines=True)

In [10]:
len(os.listdir(os.path.join(splitting2_path+'/test tables')))

996

<b>Full Tables

In [12]:
files = os.listdir(os.path.join(splitting2_path+'/test tables'))
results = []
for file in files:
    df = pd.read_json(os.path.join(os.path.join(splitting2_path+'/test tables'),file), compression='gzip', orient='records', lines=True)
    results.append(len(df))

In [17]:
print(np.mean(results))
print(np.median(results))
print(np.min(results))
print(np.max(results))

506.4668674698795
29.0
2
42723


In [15]:
files = os.listdir(os.path.join(splitting2_path+'/train tables'))
results1 = []
for file in files:
    df = pd.read_json(os.path.join(os.path.join(splitting2_path+'/train tables'),file), compression='gzip', orient='records', lines=True)
    results1.append(len(df))

In [18]:
print(np.mean(results1))
print(np.median(results1))
print(np.min(results1))
print(np.max(results1))

362.10599773670316
27.0
2
29006


<b> Not -100

In [23]:
files = os.listdir(os.path.join(splitting2_path+'/test tables'))
results_S = []
for file in files:
    df = pd.read_json(os.path.join(os.path.join(splitting2_path+'/test tables'),file), compression='gzip', orient='records', lines=True)
    df = df[df['cluster_id']!=-100]
    results_S.append(len(df))

In [24]:
print(np.mean(results_S))
print(np.median(results_S))
print(np.min(results_S))
print(np.max(results_S))

31.916666666666668
5.0
1
1587


In [25]:
files = os.listdir(os.path.join(splitting2_path+'/train tables'))
results1_S = []
for file in files:
    df = pd.read_json(os.path.join(os.path.join(splitting2_path+'/train tables'),file), compression='gzip', orient='records', lines=True)
    df = df[df['cluster_id']!=-100]
    results1_S.append(len(df))

In [26]:
print(np.mean(results1_S))
print(np.median(results1_S))
print(np.min(results1_S))
print(np.max(results1_S))

29.504715201810637
5.0
1
1319
